# Visualizing AWS JPSS data (for research)
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Provide a foundation for using JPSS data in scientific research.

This notebook is an introduction to accessing JPSS (SNPP, NOAA-20, and NOAA-21) satellite data from the AWS Cloud. It contains definitions to facilitate plotting, and provide a baseline for the analysis of, select JPSS data products. Coding experience is required to use this notebook, and it is meant to be altered to fit a researcher's data interests. 

# Importing

In [1]:
import ipywidgets as widgets
import requests
import netCDF4
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy import feature as cf
import xarray
from datetime import datetime

# Defining
#### Pre-written definitions to facilitate the visualization of select satellite data products. 

In [2]:
def aws_plot(satellite, product):
    
    fig = plt.figure(figsize=(12, 7))
    ax = plt.axes(projection=ccrs.PlateCarree()) 
    ax.add_feature(cf.OCEAN)
    ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
    ax.coastlines(resolution='50m', color='black', linewidth=0.25)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='black', alpha=0.05, linestyle='-')
    gl.top_labels = False
    gl.left_labels = True
    gl.right_labels = False
    gl.xlines = True
    gl.ylines = True
    
    # Active Fires!
    if product == "VIIRS-AF-Iband-EDR":
        for f in range(0, len(fnames)):
            fname = (fnames[f])[len(satellite) + 1:]
            s3url = (f's3://{satellite}/{fname}')
            s3f = fsspec.open(s3url, mode='rb', anon=True, default_fill_cache=False)    
            h5f = h5py.File(s3f.open(), mode='r')
            dset = h5f["Fire Pixels"]
            title = "Fire Radiative Power"
            x = dset["FP_longitude"]
            y = dset["FP_latitude"]
            product_var = dset["FP_power"]
            h5f.close()
            
            scatter = ax.scatter(x, y, s=4, c = product_var, vmin = 0, vmax = 150)
        
        plt.colorbar(scatter, shrink=0.5, label = "Megawatts")
        plt.title(f"{satellite} {title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H UTC')), loc='right')
        plt.show()
        plt.close("all")
        plt.close(fig)
    
    # Aerosol Optical Depth!
    elif product == "VIIRS-JRR-AOD":

        for f in range(0, len(fnames)):
            fname = (fnames[f])[len(satellite) + 1:]
            resp = requests.get(f'https://{satellite_choice.value}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()

            dat = dataset.metpy.parse_cf('AOD550')
            plotting_var = dataset['AOD550'].data
            lon = dat.Longitude
            lat = dat.Latitude
            dataset.close()
            
            aerosols = ax.contourf(lon, lat, plotting_var, vmin=-0.05, vmax=5, transform=ccrs.PlateCarree())
        
        title = "Aerosol Optical Depth at 550nm"
        plt.colorbar(aerosols, shrink=0.5)
        scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%SZ')

        plt.title(f"{satellite} {title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H UTC')), loc='right')
        plt.show()
        plt.close("all")
        plt.close(fig)
        
    # Aerosol Detection!
    elif product == "VIIRS-JRR-ADP":

        for f in range(0, len(fnames)):
            fname = (fnames[f])[len(satellite) + 1:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()

            dat = dataset.metpy.parse_cf('Ash')
            
            ash = dataset['Ash'].data
            ash[ash != 1] = np.nan
            
            dust = dataset['Dust'].data
            dust[dust != 1] = np.nan
            
            cloud = dataset['Cloud'].data
            cloud[cloud != 1] = np.nan
            
            smoke = dataset['Smoke'].data
            smoke[smoke != 1] = np.nan
            
            lon = dat.Longitude
            lat = dat.Latitude

            dataset.close()
            
            ash_plot = plt.contourf(lon, lat, ash, transform=ccrs.PlateCarree(), cmap = "Greens_r")
            dust_plot = plt.contourf(lon, lat, dust, transform=ccrs.PlateCarree(), cmap = "Blues_r")
            cloud_plot = plt.contourf(lon, lat, cloud, transform=ccrs.PlateCarree(), cmap = "Greys")
            smoke_plot = plt.contourf(lon, lat, smoke, transform=ccrs.PlateCarree(), cmap = "Reds_r")
            
        title = "Aerosol Detection Product"
        scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%SZ')

        h1,_ = ash_plot.legend_elements()
        h2,_ = dust_plot.legend_elements()
        h3,_ = cloud_plot.legend_elements()
        h4,_ = smoke_plot.legend_elements()
        ax.legend([h1[0], h2[0], h3[0], h4[0]], ['Ash', 'Dust', 'Cloud', 'Smoke'], loc = 'lower right')

        plt.title(f"{satellite} {title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H UTC')), loc='right')
        plt.show()
        plt.close('all')
        plt.close(fig)
    
    # Cloud Top Temperature! 
    elif product == "VIIRS-JRR-CloudHeight": 
        for f in range(0, len(fnames)):
            fname = (fnames[f])[len(satellite) + 1:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            
            dat = dataset.metpy.parse_cf('CldTopTemp')
            plotting_var = dataset['CldTopTemp'].data - 273.15
            lon = dat.Longitude
            lat = dat.Latitude
            nc.close()
            
            temps = ax.contourf(lon, lat, plotting_var, vmin = -80, vmax = 40, transform=ccrs.PlateCarree())

        title = "Cloud Top Temperature"
        plt.colorbar(temps, shrink=0.5)
        scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%SZ')

        plt.title(f"{satellite} {title}", loc='left', fontweight='bold')
        plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H UTC')), loc='right')
        plt.show()
        plt.close("all")
        plt.close(fig)

# Satellite parameters!
#### The option to customize your data search.

In [3]:
# Choose a satellite
satellite = "noaa-nesdis-n21-pds" # options are 'noaa-goes16', 'noaa-goes17', or 'noaa-goes18'

# Choose a product
product = "VIIRS-JRR-CloudHeight" 

# Choose a date
year = "2023" # must be 4 characters (i.e. "2022")
month = "11" # must be 2 characters (i.e. "01")
day = "29" # must be 2 characters (i.e. "09")

# Choose a time
shour = "23" # must be 2 characters (i.e. "01")
smin = "48" # must be 2 characters (i.e. "01")

ehour = "23" # must be 2 characters (i.e. "01")
emin = "53" # must be 2 characters (i.e. "01")

# Search data files
aws = s3fs.S3FileSystem(anon=True)
data_files = aws.ls(f'{satellite}/{product}/{year}/{month}/{day}/', refresh=True) 

# Extracting files for chosen date/time
fnames = []
for file in data_files:
    file_start = file.split('_')[-3][9:13]
    file_end = file.split('_')[-2][9:13]        
    if ((int(file_start[0:4]) >= int((str(shour) + str(smin)))) and (int(file_end[0:4]) <= int((str(ehour) + str(emin)))) and (str(file.split('_')[-2][7:9]) == day)):
        fnames.append(file)
            
print(f"This search resulted in {len(fnames)} data files.")

This search resulted in 4 data files.


# Visualize the data!

In [ ]:
# Plot single frame(s) of your data

for f in range(0, 1): 
#for f in range(0, len(fnames)):  # Uncomment and unindent to plot all images
    print(f"Plotting the first of {len(fnames)} files.")

    fname = (fnames[f])[len(satellite) + 1:]

    # Load dataset
    resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
    nc = netCDF4.Dataset(fname, memory = resp.content)

    dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
    dataset.load()
    
    aws_plot(satellite, product)

Plotting the first of 4 files.


In [ ]:
# Your analysis here!

**JPSS Product Documentation:**
- https://www.star.nesdis.noaa.gov/jpss/JPSS_products.php (Product description)
- https://www.ospo.noaa.gov/Products/Suites/jpss-rr/count_JRR_product.html?product=aerosol (JRR product visualization)
- https://www.nesdis.noaa.gov/our-satellites/currently-flying/joint-polar-satellite-system/jpss-satellite-and-instruments (About the satellites)
- https://rammb2.cira.colostate.edu/training/visit/jpss-imagery-for-users/ (Data visualization)
- https://weather.ndc.nasa.gov/sport/jpsspg/viirs.html (Real-time product visualization)

**Coding Sources:**
- https://github.com/jpss-nodd/python-scripts/tree/main
- https://medium.com/the-barometer/plotting-noaa-dnb-nighttime-data-using-python-h5py-cartopy-daefef240b0f
- https://github.com/modern-tools-workshop/AMS-python-workshop-2023
- https://www.meted.ucar.edu/education_training/course/54
- https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_level2_download.php

**CSP Access:**    
AWS: https://registry.opendata.aws/noaa-jpss/    
Google: coming soon...    
Azure: coming soon...